In [1]:
using CSV, DataFrames, JuMP, Gurobi, Plots, HDF5, JLD

In [2]:
# Some constants for the model
alpha_g1 = 0.1    # Hospitalization rate for non-vaccinated and below 65 years-old
alpha_g2 = 0.25   # Hospitalization rate for non-vaccinated and above 65 years-old
beta_g1 = 0.02 ;  # Hospitalization rate for vaccinated and below 65 years-old
beta_g2 = 0.02 ;  # Hospitalization rate for vaccinated and above 65 years-old
gamma = 0.1        # State can only vaccinate 10% of population per week
file_name = "solution_real" ; # JLD file to save the solution variables in for later analysis 

In [11]:
C

50×52 adjoint(::Matrix{Int64}) with eltype Int64:
  28929   22251   27365   26811   18659  …   3595   3272   2100   3792   4786
   2223    1724    2026    1973    1377      2919   2349   1716   1510   1219
  43988   39216   62191   67049   48815     24615  28356  20695  28655  22399
  14873   15229   19429   19328   14198      4477   3648   3879   5636   4931
 305346  271394  280190  311715  247717     36600  35113  32117  49647  43870
  17252   13697   16083   16469   11987  …  24094  20435  16604  16173  12104
  12924   12958   15791   21122   13558      4559   5262   6212  10679  18574
   4344    4508    5635    5247    4538      2239   2591   2768   4084   4392
  74172   75332   83762  102769   97100     10694  11384   9998  11389  14269
  47684   46452   59099   68454   54424      6959   7526   6626   9338  10882
    897     687    1131    1362     889  …    689    673    600    728   1565
   8533    5880    6245    6255    5223      3209   3287   2432   3099   2380
  47465   3731

In [12]:
data_folder = "real_world_data"
# Load csv files
case_file_g1 = data_folder * "/" * "case_data_g1.csv"
case_file_g2 = data_folder * "/" * "case_data_g2.csv"
population_file = data_folder * "/" * "population_data_age.csv"
vaccine_file = data_folder * "/" * "vaccine_data.csv"

case_data_g1 = CSV.read(case_file_g1, DataFrame)
case_data_g2 = CSV.read(case_file_g2, DataFrame)
population_data = CSV.read(population_file, DataFrame)
vaccine_data = CSV.read(vaccine_file, DataFrame)


P = Matrix(population_data)
C1 = Matrix(case_data_g1)
C2 = Matrix(case_data_g2)
V = Matrix(vaccine_data)
# convert population data to matrix

C1 = C1'
C2 = C2'
V = V[:,1]
P = P[:,2:3]

println("C1: ", size(C1))
println("C2: ", size(C2))
println("P: ", size(P))
println("V: ", size(V))

LoadError: ArgumentError: "real_world_data/case_data_g1.csv" is not a valid file or doesn't exist

In [ ]:
n_states = 1:size(case_data, 2)
n_weeks = 1:size(case_data, 1) 
n_weeks_2 = 1:size(case_data, 1)-1 

total_states = length(n_states) 
total_weeks = length(n_weeks) 
println("Number of states: ", n_states)
println("Number of weeks: ", n_weeks)

In [ ]:
model = Model(Gurobi.Optimizer)

@variable(model, X1[n_states, n_weeks] >= 0);
@variable(model, X2[n_states, n_weeks] >= 0);
@variable(model, W1[n_states, n_weeks] >= 0);
@variable(model, W2[n_states, n_weeks] >= 0);
@variable(model, Y[n_states, n_weeks] >= 0);

@objective(model, Min, sum(sum(Y[i,j] for i in n_states) for j in n_weeks));

@constraint(model, hospitalizations[i in n_states,j in n_weeks], Y[i,j] == alpha_g1 * C[i,j] * (1 - W[i,j]) + beta * C[i,j] * W[i,j]) 
@constraint(model, vaccinated[i in n_states,j in n_weeks_2], W[i,j+1] == W[i,j] + X[i,j]/ P[i] ) 

In [ ]:
# Initial values of W are  0 
@constraint(model, initial[i in n_states], W[i,1] == 0) 

In [ ]:
# W should be between 0 and 1
@constraint(model, bounds[i in n_states,j in n_weeks], W[i,j] <= 1)

In [ ]:
@constraint(model,vaccine_supply[j in n_weeks], sum(X[i,j] for i in n_states) <= V[j]) 

In [ ]:
@constraint(model, state_vaccination_capacity[i in n_states, j in n_weeks], X[i,j] <= gamma * P[i])  

In [ ]:
optimize!(model)

In [ ]:
objective_value(model)


In [ ]:
W = value.(W)
X = value.(X)
Y = value.(Y)

X_soln = zeros(total_states, total_weeks)
Y_soln = zeros(total_states, total_weeks)
W_soln = zeros(total_states, total_weeks)
# Compute distances between all pairs of points
for i in n_states
    for j in n_weeks
        X_soln[i,j] = X[i,j]
        Y_soln[i,j] = Y[i,j]
        W_soln[i,j] = W[i,j]
    end
end

# save the solutions as a jld file
save("solution_real.jld", "X_soln", X_soln, "Y_soln", Y_soln, "W_soln", W_soln)

In [ ]:
print(Y)

In [ ]:
us_cases = CSV.read("real_original_data/us_cases.csv", DataFrame)